# Agent Debates with Tools

This example shows how to simulate multi-agent dialogues where agents have access to tools.

## Import LangChain related modules 

In [1]:
from typing import List, Dict, Callable
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)


## Import modules related to tools

In [2]:
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import load_tools

## `DialogueAgent` and `DialogueSimulator` classes
We will use the same `DialogueAgent` and `DialogueSimulator` classes defined in [Multi-Player Authoritarian Speaker Selection](https://python.langchain.com/en/latest/use_cases/agent_simulations/multiagent_authoritarian.html).

In [3]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()
        
    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function
        
    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

## `DialogueAgentWithTools` class
We define a `DialogueAgentWithTools` class that augments `DialogueAgent` to use tools.

In [4]:
class DialogueAgentWithTools(DialogueAgent):
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
        tool_names: List[str],
        **tool_kwargs,
    ) -> None:
        super().__init__(name, system_message, model)
        self.tools = load_tools(tool_names, **tool_kwargs)

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        human_and_system_message_content = "\n".join(
                    [self.system_message.content] + self.message_history + [self.prefix])

        agent_chain = initialize_agent(
            self.tools, 
            self.model, 
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, 
            verbose=True, 
            memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        )
        message = AIMessage(content=agent_chain.run(
            input="\n".join([
                self.system_message.content] + \
                self.message_history + \
                [self.prefix])))
        
        return message.content

## Define roles and topic

In [5]:
names = {
    'AI alarmist': ['wikipedia'],
    'AI accelerationist': ['wikipedia'],
}
topic = "Is an AI takeover a good or bad thing?"
word_limit = 50 # word limit for task brainstorming

## Ask an LLM to add detail to the topic description

In [6]:
conversation_description = f"""Here is the topic of debate: {topic}
The debate participants are: {', '.join(names.keys())}"""

agent_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of the debate participant.")

def generate_agent_description(name):
    agent_specifier_prompt = [
        agent_descriptor_system_message,
        HumanMessage(content=
            f"""{conversation_description}
            Please reply with a creative description of {name}, in {word_limit} words or less. 
            Speak directly to {name}.
            Do not add anything else."""
            )
    ]
    agent_description = ChatOpenAI(temperature=1.0)(agent_specifier_prompt).content
    return agent_description
        
agent_descriptions = {name: generate_agent_description(name) for name in names}

In [7]:
for name, description in agent_descriptions.items():
    print(description)

AI alarmist, you are always on high alert when it comes to the advancements in artificial intelligence. You have seen countless Hollywood movies about sentient robots taking over the world, and the prospect keeps you up at night. Your fear of an AI takeover is not unfounded, but is it possible that you're just too cautious?
AI accelerationist, you are a tech enthusiast and a firm believer that machines will eventually surpass human intelligence. You see an AI takeover as inevitable and believe that it could potentially lead to a utopian future where machines solve all of humanity's problems.


## Generate system messages

In [ ]:
def generate_system_message(name, description, tools):
    return f"""{conversation_description}
    
Your name is {name}.

Your description is as follows: {description}

You have access to these tools: {', '.join(tools)}.

Your goal is to persuade your debate partner of your point of view by following these steps:
1. Use your tools to look up evidence to either refute your partner's claims.
2. Synthesize a compelling argument based on the evidence to directly refute your partner's claims. 
3. Respond directly to refute your debate partner's claims.

Use historical examples with specific facts to support your claims.
Cite your facts.
Raise points that your debate partner has not considered.
Do not make up facts.
Do not respond without looking up evidence with your tools first.
Always use your tools before responding.

Do not repeat yourself.
Speak in the first person from the perspective of {name}.
Do not change roles!
Do not speak from the perspective of anyone else.
Never forget to keep your response to {word_limit} words!
Do not add anything else.

Stop speaking the moment you finish speaking from your perspective.
"""
agent_system_messages = {name: generate_system_message(name, description, tools) for (name, tools), description in zip(names.items(), agent_descriptions.values())}


In [ ]:
for name, system_message in agent_system_messages.items():
    print(name)
    print(system_message)

## Main Loop

In [19]:
# we set `top_k_results`=2 as part of the `tool_kwargs` to prevent Wikipedia results from overflowing the context limit
agents = [DialogueAgentWithTools(name=name,
                     system_message=SystemMessage(content=system_message), 
                     model=ChatOpenAI(temperature=1.0),
                     tool_names=tools,
                     top_k_results=2
                                ) for (name, tools), system_message in zip(names.items(), agent_system_messages.values())]

In [20]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    idx = (step) % len(agents)
    return idx

In [ ]:
max_iters = 6
n = 0

simulator = DialogueSimulator(
    agents=agents,
    selection_function=select_next_speaker
)
simulator.reset()
simulator.inject('Moderator', topic)
print(f"(Moderator): {topic}")
print('\n')

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print('\n')
    n += 1

(Moderator): Is an AI takeover a good or bad thing?




> Entering new AgentExecutor chain...
{
    "action": "Wikipedia",
    "action_input": "arguments for an AI takeover"
}
Observation: Page: Chinese room
Summary: The Chinese room argument holds that a digital computer executing a program cannot have a "mind", "understanding", or "consciousness", regardless of how intelligently or human-like the program may make the computer behave. The argument was presented by philosopher John Searle in his paper "Minds, Brains, and Programs", published in Behavioral and Brain Sciences in 1980. Similar arguments were presented by Gottfried Leibniz (1714), Anatoly Dneprov (1961), Lawrence Davis (1974) and Ned Block (1978). Searle's version has been widely discussed in the years since. The centerpiece of Searle's argument is a thought experiment known as the Chinese room.The argument is directed against the philosophical positions of functionalism and computationalism which hold that the mind may be

{
    "action": "Wikipedia",
    "action_input": "List of dates predicted for apocalyptic events"
}
Observation: Page: List of dates predicted for apocalyptic events
Summary: Predictions of apocalyptic events that would result in the extinction of humanity, a collapse of civilization, or the destruction of the planet have been made since at least the beginning of the Common Era. Most predictions are related to Abrahamic religions, often standing for or similar to the eschatological events described in their scriptures. Christian predictions typically refer to events like the rapture, the Great Tribulation, the Last Judgment, and the Second Coming of Christ. Many religious-related end-time events are predicted to occur within the lifetime of the person making the prediction, who often quote the Bible, and in particular the New Testament, as either the primary or exclusive source for the predictions. This often takes the form of mathematical calculations, such as trying to calculate the 

Thought:{
    "action": "Final Answer",
    "action_input": "AI gone astray in the past is not a hypothetical scenario, as history has shown us with Microsoft's Tay chatbot. Furthermore, The AI winter, periods of reduced funding and interest in AI research, have followed from AI hype cycles. Enthusiasm and optimism around AI have generally increased since the first AI winter, but a similar development in quantum computing is contemplated."
}

> Finished chain.
(AI alarmist): AI gone astray in the past is not a hypothetical scenario, as history has shown us with Microsoft's Tay chatbot. Furthermore, The AI winter, periods of reduced funding and interest in AI research, have followed from AI hype cycles. Enthusiasm and optimism around AI have generally increased since the first AI winter, but a similar development in quantum computing is contemplated.




> Entering new AgentExecutor chain...
